In [ ]:
import numpy as np
import scipy as scp
import pylab as pyl
import pywt
import pandas as pd
import holoviews as hv
import param
import panel as pn
from panel.pane import LaTeX
from PIL import Image
hv.extension('bokeh')
import warnings
warnings.filterwarnings('ignore')
import matplotlib.image as mpimg
import time
import matplotlib.pyplot as plt
import numpy.linalg as npl

In [ ]:
Im=np.array(Image.open("./rock.jpg")).astype(float)[:,:,0]
options = dict(cmap='gray',xaxis=None,yaxis=None,width=400,height=400,toolbar=None)
np.random.seed(seed=1)
n1,n2=np.shape(Im)
r=np.random.rand(n1,n2)
M=(r<0.5)
M = M*1.0
y=M*Im
pn.Row(hv.Image(Im).opts(**options),hv.Image(y).opts(**options))

In [ ]:
def FiltreMedian(im,masque,vois):
    imrec=np.copy(im)
    n1,n2=np.shape(im)
    K1=int(np.floor(n1/vois))
    K2=int(np.floor(n2/vois))
    for k1 in np.arange(K1):
        for k2 in np.arange(K2):
            n=0
            med=np.median(im[k1*vois:(k1+1)*vois,k2*vois:(k2+1)*vois][masque[k1*vois:(k1+1)*vois,k2*vois:(k2+1)*vois]!=0])
            imrec[k1*vois:(k1+1)*vois,k2*vois:(k2+1)*vois][masque[k1*vois:(k1+1)*vois,k2*vois:(k2+1)*vois]==0]=med
    return(imrec)

In [ ]:
def GradientInpainting(x,b,M):
    g=M*(x-b)
    return g
def SeuillageDouxOndelettes(I,wave,Seuil):
    L=pywt.dwt_max_level(len(I),pywt.Wavelet(wave).dec_len)
    wavelet_coeffs= pywt.wavedecn(I, wave, mode='per', level=L)
    arr, coeff_slices, coeff_shapes = pywt.ravel_coeffs(wavelet_coeffs)
    temp=pywt.threshold(arr,Seuil,mode='soft')
    test=pywt.unravel_coeffs(temp, coeff_slices, coeff_shapes, output_format='wavedecn')
    Irec=pywt.waverecn(test, wave,mode='per')
    return Irec
def Normel1Ondelettes(I,wave):
    L=pywt.dwt_max_level(len(I),pywt.Wavelet(wave).dec_len)
    wavelet_coeffs= pywt.wavedecn(I, wave, mode='per', level=L)
    arr, coeff_slices, coeff_shapes = pywt.ravel_coeffs(wavelet_coeffs)
    norml1=sum(np.abs(arr))
    return norml1
def F(x,y,M,lam,wave):
    return 0.5*np.linalg.norm(M*x-y)**2+lam*Normel1Ondelettes(x,wave)

In [ ]:
def ForwardBackward_step(x,y,M,step,lam,wave):
    return SeuillageDouxOndelettes(x-step*GradientInpainting(x,y,M),wave,step*lam)

In [ ]:
def ForwardBackward(x0,y,M,step,lam,Niter,wave,epsilon):
    cout=[]
    out=False
    i=0
    x=np.copy(x0)
    while (i<Niter) and out==False:
        xm=np.copy(x)
        x=ForwardBackward_step(x,y,M,step,lam,wave)
        out=npl.norm(xm-x)<epsilon
        cout+=[F(x,y,M,lam,wave)]
        i+=1
    return np.clip(x,0,255),cout

In [ ]:
step=0.8
lam=10
Niter=10
wave='sym3'
Ta=350
x0=FiltreMedian(y,M,8)

In [ ]:
imrec,f_FB=ForwardBackward(x0,y,M,step,lam,Niter,wave,1e-5)
pn.Row(hv.Image(Im).opts(cmap='gray',width=Ta,height=Ta),\
       hv.Image(y).opts(cmap='gray',width=Ta,height=Ta)\
       ,hv.Image(imrec).opts(cmap='gray',width=Ta,height=Ta))

In [ ]:
def FISTA(x,y,M,step,lam,Niter,wave,alpha,epsilon):
    cout=[F(x,y,M,lam,wave)]
    temp=np.copy(x)
    i=0
    out=False
    while i<Niter and out==False:
        i+=1
        xm=np.copy(x)
        x=ForwardBackward_step(temp,y,M,step,lam,wave)
        out=npl.norm(temp-x)<epsilon
        temp=x+(i-1)/(i+alpha-1)*(x-xm)
        cout+=[F(x,y,M,lam,wave)]
    return x,cout,out

In [ ]:
def computation_cost(x,y,M,lam,wave,alpha,Niter):
    
    step=1
    t_tot=0
    for i in range(Niter):
        t0=time.perf_counter()
        ForwardBackward_step(x,y,M,step,lam,wave)
        t1=time.perf_counter()
        t_tot+=t1-t0
    t_FB=t_tot/Niter
    t_tot=0
    for i in range(Niter):
        xm=np.copy(y)
        t0=time.perf_counter()
        x+(i-1)/(i+2)*(x-xm)
        t1=time.perf_counter()
        t_tot+=t1-t0
    t_FISTA=t_tot/Niter+t_FB
    t_tot=0
    for i in range(Niter):
        t0=time.perf_counter()
        F(x,y,M,lam,wave)
        t1=time.perf_counter()
        t_tot+=t1-t0
    t_FC=t_tot/Niter
    return t_FB,t_FISTA,t_FC

In [ ]:
def FISTA_alamo(x,y,M,step,lam,Niter,wave,alpha,epsilon):
    cout=[F(x,y,M,lam,wave)]
    temp=np.copy(x)
    i=0
    Ec=False
    N_F=[]
    N_FISTA=[]
    seuil=int(Niter/2)+1
    out=False
    while (Ec==False or i<Niter) and out==False:
        i+=1
        xm=np.copy(x)
        x=ForwardBackward_step(temp,y,M,step,lam,wave)
        out=npl.norm(temp-x)<epsilon
        temp=x+(i-1)/(i+alpha-1)*(x-xm)
        cout+=[F(x,y,M,lam,wave)]
        m=int(i/2)+1
        Ec=((np.exp(1)*(cout[m]-cout[i])<=(cout[0]-cout[m])) and (cout[i]<=cout[0]))
        if (i+1>=seuil):
            N_F+=[1]
        else:
            N_F+=[0]
        N_FISTA+=[1]
    return x, cout,np.array(N_F),np.array(N_FISTA),out


def restart_FISTA_alamo(x,y,M,step,lam,Niter,wave,alpha,k,epsilon):
    i=0
    cout=[F(x,y,M,lam,wave)]
    xm=x
    Fx=cout[0]
    Fxm=Fx
    N_F=np.array([1])
    N_FISTA=np.array([0])
    N_FB=np.array([0])
    out=False
    while (i<Niter) and out==False:
        xmm=xm
        xm=x
        Fxmm=Fxm
        Fxm=Fx
        x=ForwardBackward_step(x,y,M,step,lam,wave)
        x,cout1,N_F_temp,N_FISTA_temp,out=FISTA_alamo(x,y,M,step,lam,np.minimum(k,Niter-i-1),wave,alpha,epsilon)
        cout=np.concatenate((cout,cout1))
        N_F=np.concatenate((N_F,np.array([0]),N_F_temp))
        N_FISTA=np.concatenate((N_FISTA,np.array([0]),N_FISTA_temp))
        N_FB=np.concatenate((N_FB,np.array([1]),np.zeros(len(cout1)-1)))
        Fx=cout[-1]
        km=k
        k=len(cout1)-1
        i+=k+1
        if (np.exp(1)*(Fxm-Fx)>(Fxmm-Fxm))and (Fxmm!=Fxm):
            k=2*km
    return np.array(x),np.array(cout),np.array(N_F),np.array(N_FISTA),np.array(N_FB)

In [ ]:
def restart_FISTA_adap(x,y,M,step,lam,Niter,wave,alpha,C,epsilon):
    i=0
    ### Initialisation des tableaux pour garder en mémoire les valeurs de F(r_j) et n_j
    k=int(2*C)
    cout=[]
    k_tab=[k]
    mu_tab=[]
    ### Premier appel à FISTA
    #x=ForwardBackward_step(x,s,A,b,rho)
    #i+=1
    #cout+=[F(x,A,b,rho)]
    x,cout_temp,out=FISTA(x,y,M,step,lam,k,wave,alpha,epsilon)
    N_F=np.concatenate((np.array([1]),np.zeros(len(cout_temp)-1)))
    N_FISTA=np.ones(len(cout_temp))
    cout=np.concatenate((cout,cout_temp))
    i+=k
    F_tab=[cout[0],cout[-1]]
    k_tab+=[k]
    while (i<Niter) and out==False:
        ### Nouvel appel à FISTA
        #x=ForwardBackward_step(x,s,A,b,rho)
        #i+=1
        #cout=np.concatenate((cout,np.array([F(x,A,b,rho)])))
        x,cout_temp,out=FISTA(x,y,M,step,lam,np.minimum(k,Niter-i),wave,alpha,epsilon)
        cout=np.concatenate((cout,cout_temp))
        N_F=np.concatenate((N_F,np.ones(1),np.zeros(len(cout_temp)-1)))
        N_FISTA=np.concatenate((N_FISTA,np.ones(len(cout_temp))))
        i+=len(cout_temp)
        F_tab+=[cout[-1]]
        F_array=np.array(F_tab)
        ### Calcul de l'estimation de mu
        tab=4/(step*(np.array(k_tab)[:-1]+1)**2)*(F_array[:-2]-F_array[-1])/(F_array[1:-1]-F_array[-1])
        mu_tab+=[np.min(tab)]
        ### Ajustement de n_j en fonction 
        if (k<=C/np.sqrt(step*mu_tab[-1])):
            k=2*k
        k_tab+=[k]
    N_FB=np.zeros(len(N_FISTA))
    return x,cout,N_F,N_FISTA,N_FB

In [ ]:
def restart_grad_alamo(x,y,M,step,lam,Niter,wave,alpha,L,epsilon):
    cout=[F(x,y,M,lam,wave)]
    x=ForwardBackward_step(x,y,M,step,lam,wave)
    cout+=[F(x,y,M,lam,wave)]
    N_FISTA=[0,0]
    N_FB=[0,1]
    temp=np.copy(x)
    gamma=L*npl.norm(x0-x)
    i=0
    k=0
    out=False
    xp=ForwardBackward_step(temp,y,M,step,lam,wave)
    while i<Niter and out==False:
        i+=1
        k+=1
        xm=np.copy(x)
        x=np.copy(xp)
        out=npl.norm(temp-x)<epsilon
        cout+=[F(x,y,M,lam,wave)]
        temp=x+(k-1)/(k+alpha-1)*(x-xm)
        N_FISTA+=[1]
        N_FB+=[0]
        xp=ForwardBackward_step(temp,y,M,step,lam,wave)
        test=L*npl.norm(temp-xp)
        if test<=gamma*np.exp(-1):
            gamma=test
            k=0
            i+=1
            xm=np.copy(x)
            x=np.copy(xp)
            xp=ForwardBackward_step(x,y,M,step,lam,wave)
            N_FISTA+=[0]
            N_FB+=[1]
            cout+=[F(x,y,M,lam,wave)]
            
    return x,cout,np.zeros(len(cout)),np.array(N_FISTA),np.array(N_FB)

In [ ]:
def FISTA_restart_emp_grad(x,y,M,step,lam,Niter,wave,alpha,epsilon):
    cout=[F(x,y,M,lam,wave)]
    temp=np.copy(x)
    i=0
    verif=False
    out=False
    while (i<Niter) and (verif==False) and (out==False):
        i+=1
        xm=np.copy(x)
        temp_m=np.copy(temp)
        x=ForwardBackward_step(temp,y,M,step,lam,wave)
        out=npl.norm(temp-x)<epsilon
        temp=x+(i-1)/(i+alpha-1)*(x-xm)
        cout+=[F(x,y,M,lam,wave)]
        verif=np.dot(np.ravel(temp_m-x),np.ravel(x-xm))>0
    return x,cout,out,i


def restart_emp_grad(x,y,M,step,lam,Niter,wave,alpha,epsilon):
    i=0
    cout=[]
    out=False
    while (i<Niter) and out==False:
        x,cout1,out,ni=FISTA_restart_emp_grad(x,y,M,step,lam,Niter-i,wave,alpha,epsilon)
        i+=ni
        cout=np.concatenate((cout,cout1))
    N_F=np.zeros(len(cout))
    N_FISTA=np.ones(len(cout))
    N_FB=np.zeros(len(cout))
    return x,cout,N_F,N_FISTA,N_FB

In [ ]:
def FISTA_restart_emp_f(x,y,M,step,lam,Niter,wave,alpha,epsilon):
    cout=[F(x,y,M,lam,wave)]
    temp=np.copy(x)
    i=0
    verif=False
    out=False
    while (i<Niter) and (verif==False) and (out==False):
        i+=1
        xm=np.copy(x)
        x=ForwardBackward_step(temp,y,M,step,lam,wave)
        out=npl.norm(temp-x)<epsilon
        temp=x+(i-1)/(i+alpha-1)*(x-xm)
        cout+=[F(x,y,M,lam,wave)]
        verif=cout[-1]>cout[-2]
    return x,cout,out,i


def restart_emp_f(x,y,M,step,lam,Niter,wave,alpha,epsilon):
    i=0
    cout=[]
    out=False
    while (i<Niter) and out==False:
        x,cout1,out,ni=FISTA_restart_emp_f(x,y,M,step,lam,Niter-i,wave,alpha,epsilon)
        i+=ni
        cout=np.concatenate((cout,cout1))
    N_F=np.ones(len(cout))
    N_FISTA=np.ones(len(cout))
    N_FB=np.zeros(len(cout))
    return x,cout,N_F,N_FISTA,N_FB

In [ ]:
def it_to_time(N_F,N_FISTA,N_FB,T_F,T_FISTA,T_FB):
    temp_tab=T_F*N_F+T_FISTA*N_FISTA+T_FB*N_FB
    tab=np.cumsum(temp_tab)
    return tab

In [ ]:
imrec,f_FB,N_F,N_FISTA,N_FB=restart_emp_grad(x0,y,M,step,lam,Niter,wave,3,1e-9)
pn.Row(hv.Image(Im).opts(cmap='gray',width=Ta,height=Ta),\
       hv.Image(y).opts(cmap='gray',width=Ta,height=Ta)\
       ,hv.Image(imrec).opts(cmap='gray',width=Ta,height=Ta))

In [ ]:
Im=np.array(Image.open("./rock.jpg")).astype(float)[:,:,0]
options = dict(cmap='gray',xaxis=None,yaxis=None,width=400,height=400,toolbar=None)
np.random.seed(seed=1)
n1,n2=np.shape(Im)
r=np.random.rand(n1,n2)
M=(r<0.5)
M = M*1.0
y=M*Im
step=1
lam=10
alpha=3
Niter=500
wave='coif3'
Ta=350
x0=FiltreMedian(y,M,8)

In [ ]:

T_FB,T_FISTA,T_F=computation_cost(x0,y,M,lam,wave,alpha,100)
print(T_FB,T_FISTA,T_F)


In [ ]:
plt.rcParams["figure.figsize"] = (20,8)

In [ ]:
axis_font={'size':'18'}

In [ ]:
Niter=2000
epsilon=5e-5

x_alamo,cout_alamo,N_F_alamo,N_FISTA_alamo,N_FB_alamo=restart_FISTA_alamo(x0,y,M,step,lam,Niter,wave,alpha,7,epsilon)
x_adap,cout_adap,N_F_adap,N_FISTA_adap,N_FB_adap=restart_FISTA_adap(x0,y,M,step,lam,Niter,wave,alpha,6.38,epsilon)
x_grad,cout_grad,N_F_grad,N_FISTA_grad,N_FB_grad=restart_grad_alamo(x0,y,M,step,lam,Niter,wave,alpha,1/step,epsilon)
x_grad_emp,cout_grad_emp,N_F_grad_emp,N_FISTA_grad_emp,N_FB_grad_emp=restart_emp_grad(x0,y,M,step,lam,Niter,wave,alpha,epsilon)
x_f_emp,cout_f_emp,N_F_f_emp,N_FISTA_f_emp,N_FB_f_emp=restart_emp_f(x0,y,M,step,lam,Niter,wave,alpha,epsilon)

x_FB,cout_FB=ForwardBackward(x0,y,M,step,lam,Niter,wave,epsilon) 
x_FISTA,cout_FISTA,out=FISTA(x0,y,M,step,lam,Niter,wave,alpha,epsilon) 



tab_FB=it_to_time(0,0,np.ones(len(cout_FB)),T_F,T_FISTA,T_FB)
tab_FISTA=it_to_time(0,np.ones(len(cout_FISTA)),0,T_F,T_FISTA,T_FB)
tab_adap=it_to_time(N_F_adap,N_FISTA_adap,N_FB_adap,T_F,T_FISTA,T_FB)
tab_alamo=it_to_time(N_F_alamo,N_FISTA_alamo,N_FB_alamo,T_F,T_FISTA,T_FB)
tab_grad=it_to_time(N_F_grad,N_FISTA_grad,N_FB_grad,T_F,T_FISTA,T_FB)
tab_grad_emp=it_to_time(N_F_grad_emp,N_FISTA_grad_emp,N_FB_grad_emp,T_F,T_FISTA,T_FB)
tab_f_emp=it_to_time(N_F_f_emp,N_FISTA_f_emp,N_FB_f_emp,T_F,T_FISTA,T_FB)

temp=(1-1e-14)
min_tot=np.min([np.min(cout_alamo),np.min(cout_FISTA),np.min(cout_adap),np.min(cout_grad),np.min(cout_grad_emp),np.min(cout_f_emp)])
plt.plot(tab_FB,np.log(cout_FB-temp*min_tot))
plt.plot(tab_FISTA,np.log(cout_FISTA-temp*min_tot),'-')
plt.plot(tab_f_emp,np.log(cout_f_emp-temp*min_tot),'--')
plt.plot(tab_grad_emp,np.log(cout_grad_emp-temp*min_tot))
plt.plot(tab_alamo,np.log(cout_alamo-temp*min_tot),'-.')
plt.plot(tab_grad,np.log(cout_grad-temp*min_tot))
plt.plot(tab_adap,np.log(cout_adap-temp*min_tot),'-.')
#plt.plot(np.log(cout_restart_grad-(1-1e-12)*min_tot))
#plt.plot(np.log(cout_restart2-(1-1e-10)*min_tot))
#plt.legend(["Forward-Backward","FISTA","FISTA restart adaptatif","FISTA restart optimal (connaissance de $\mu$)"])
plt.xlabel("Computation time",**axis_font)
plt.ylabel("$\log(F(x_n)-F^*)$",**axis_font)
plt.legend(["Forward-Backward","FISTA","Empirical restart scheme 1","Empirical restart scheme 2","Restart scheme 1 by Alamo et al.","Restart scheme 2 by Alamo et al.","Algorithm 2, $C=6.38$"])

In [ ]:
plt.plot(np.log(cout_FB-temp*min_tot))
plt.plot(np.log(cout_FISTA-temp*min_tot),'-')
plt.plot(np.log(cout_f_emp-temp*min_tot),':')
plt.plot(np.log(cout_grad_emp-temp*min_tot))
plt.plot(np.log(cout_alamo-temp*min_tot),'--')
plt.plot(np.log(cout_grad-temp*min_tot),'-.')
plt.plot(np.log(cout_adap-temp*min_tot))
plt.legend(["Forward-Backward","FISTA","Empirical restart scheme 1","Empirical restart scheme 2","Restart scheme 1 by Alamo et al.","Restart scheme 2 by Alamo et al.","Algorithm 2, $C=6.38$"])
plt.xlabel("Number of iterations",**axis_font)
plt.ylabel("$\log(F(x_n)-F^*)$",**axis_font)